In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Tarea 1 - Momentos Estadísticos en Series de Tiempo Financieras
### Diego Andrés Peñaranda Correa 201910807
1. Descarga de la serie de precios de dos activos: ETF del S&P 500: (SPY) y 1 Bond ETF: (BND).
Fechas: 1 enero 2015 al 1 31 Dic 2018.
2. Análisis de las series:
Media, Varianza, Sesgo y Curtosis de rendimientos simples.
Pruebas de hipótesis de cero: media, sesgo, y curtosis. 
Prueba de normalidad JB o Shapiro.
Con base a los resultados, comente y compare las series entre ellas:
¿Cuál es más y menos riesgosa, cuál puede presentar mayores pérdidas y por qué?.
Comenta si identificas que el tipo de distribuciones (leptocúrticas, platicpúrticas, etc) están presentes en alguna de las series. Si sí, cómo interpretas esto respecto al tipo de activo.

### 1. Parte 1: Obtención de datos y grafico de rendimientos.
### 2. Estaísticas Basicas y Análisis Momentos: media, sesgo y curtosis.
### 3. Visualización de distribuciones

In [ ]:
library(fBasics)
library(PerformanceAnalytics)
library(xts)
library(quantmod)
library(ggplot2)
library(tseries)
library(dygraphs)
options(warn = - 1)

## 1. Datos
Se toman los datos con la herramienta getSymbols desde la base de datos de Yahoo Finance, se tiene en cuenta que los rendimientos diarios sean simples. Las fechas en el intervalo del 1 enero 2015 al 31 Dic 2018.

In [ ]:
start<-format(as.Date("2015-01-01"),"%Y-%m-%d")
end<-format(as.Date("2018-12-31"),"%Y-%m-%d")
rend<-function(simbolo) {
  ##---------Obtener precios de yahoo finance:
  datos<-getSymbols(simbolo,  from=start, to= end, auto.assign = FALSE)
  ##---------eliminar datos faltantes:
  datos<-na.omit(datos)
  ##--------Mantener el precio de interés:
  datos<-datos[,4]
  ##--------Rendimientos simples:
  rend<-periodReturn(datos, period = "daily", type='arithmetic')                                              
  #------ --Para hacer dtos accesibles  GLobal ENv:
  assign(simbolo, rend, envir = .GlobalEnv)
}

In [ ]:
rend("SPY") #ETF del S&P 500: (SPY)
rend("BND")  #Bond ETF: (BND)

head(SPY) #primeros cinco datos del ETF
head(BND) #primeros cinco datos de la serie del Bono

In [ ]:
rends<-merge.xts(SPY, BND)
colnames(rends)<-c("SPY", "BND")
dygraph(rends, main = "SPY, BND Rendimientos") %>%
  dyAxis("y", label = "Rend %") %>%
  dyOptions(colors = RColorBrewer::brewer.pal(4, "Set1"))

Podemos ver de la gráfica de rendimientos que la serie del S&P 500 para el periodo evaluado es más volatil que el bono ETF, esto entre todo el perido de evaluación. Esto principalmente se puede justificar por la variedad de activos riesgosos y estables que componen el portafolio de ún índice como el S&P 500, mientras que los bonos son instrumentos financieros con mayor estabilidad.

## 2. Estadísticas y análisis momentos

Para medir estadísticamente la significancia del sesgo, se usan las pruebas de hipótesis basadas en las funciones en R:

Donde en el caso de curtosis, el programa calcula el exceso de curtosis. Inicialmente se calculan los primeros momentos, además se define si la media de los rendimientos es o no igual a cero:


### Análisis del S&P 500:
* rend("SPY")
### Además, se incluirán conclusiones del nivel de riesgo del índice

In [ ]:
basicStats(SPY) ## Resumen estadisticos

In [ ]:
mean(SPY)
var(SPY)
stdev(SPY) # Desv Std

A continuación, se evalua si la media de los rendimientos es igual o diferente de cero, esto a un nivel de significación alfa=0.05.

In [ ]:
t.test(SPY)  # Prueba que H0: media real = 0

Se encuentra que el p-value es mayor al nivel de significancia, por tanto, la prueba esta en zona de no rechazo y la hipotesis nula inicial (H0) no se puede rechazar. En resumen, la media real es cero. Continuamos con la prueba de sesgo o tercer momento:

In [ ]:
s3=skewness(SPY)  #Sesgo
T=length(SPY) # N
t3=s3/sqrt(6/T) # Prueba de sesgo
t3

El sesgo es negativo. Ahora se determina la significancia de dicho sesgo con prueba de hipótesis mediante el cálculo del p-valor:

In [ ]:
pp=2*pt(abs(t3), T-1, lower=FALSE) # H0: Sesgo=0 
pp

Como el pp es menor al nivel de significancia 0.05 se encuentra la H0 en zona de rechazo, por lo cual se puede concluir que el sesgo es estadísticamente diferente de cero. 
Ahora validamos en el caso de la curtosis:

In [ ]:
s4=kurtosis(SPY)
s4
t4=s4/sqrt(24/T) # Prueba de curtosis, en exceso
t4
pv=2*(1-pnorm(t4)) # p-valor, H0:curtosis=0
pv

Al ser el p-valor de esta prueba menor al alpha de significancia se puede rechazar la hipotesis nula, es decir, la curtosis es distinta de cero. Finalmente, se determina si la distribución es o no normal. Podemos usar Jarque Bera o Shapiro. En este caso usamos JB:

In [ ]:
normalTest(SPY,method='jb') # Prueba Jaque Bera, H0: Normal

El p-value resultante esta dentro de la zona de rechazo, por lo cual se rechaza H0, es decir, que la distribución no es normal.

Ahora graficamos el comportamiento de la distribución para esta acción:

In [ ]:
library(PerformanceAnalytics)
par(mfrow=c(1,1))
chart.Histogram(SPY, methods = c("add.normal", "add.density"), colorset = c("gray", "blue", "red"))
legend("topright", legend = c("Hist-SPY" ,"S&P 500 dist","dnorm S&P 500"), col=c("gray", "blue", "red"), lty=1, cex = 0.7)

Se puede concluir a partir de esta visualización que el índice SPY presenta una gráfica con comportamiento leptocúrtico, con una media de rendimientos estadísticamente igual a cero, sesgada negativamente (aunque levemente) y con un exceso de curtosis positivo significativo. Así se puede interpretar como una acción riesgosa. Adicionalmente por ser un índice de mercado es altamente sensible a la variedad de mercados que componen las acciones de su portafolio, esto genera una alta volatilidad y riesgo, este factor puede explicar complementariamente el comportamiento evidenciado.

### A continuación se evaluan los momentos y se hace el análisis para el bono ETF (BND):
* rend("BND")
### Además, se incluirán conclusiones del nivel de riesgo del bono analizado

In [ ]:
basicStats(BND) ## Resumen estadisticos
mean(BND)
var(BND)
stdev(BND) # Desv Std

In [ ]:
t.test(BND)  # Prueba que H0: mean return = 0

Como resultante el p-value de esta prueba esta en zona de no rechazo, por lo cual no se puede rechazar la H0, es decir, la media es estadísticamente igual a 0. Se continua ahora con el sesgo:

In [ ]:
s3=skewness(BND)  #Sesgo
T=length(BND) # N
t3=s3/sqrt(6/T) # Prueba de sesgo
t3

En este caso lo relevante de este resultado es que el sesgo es negativo. Se sigue ahora con la prueba de hipotesis del sesgo.

In [ ]:
pp=2*pt(abs(t3), T-1, lower=FALSE) # H0: Sesgo=0 
pp

Dado que el p-value para la prueba de sesgo es menor a un nivel de significancia del 5% se encuentra que dicho p-value obtenido está en zona de rechazo, es decir, se puede rechazar la H0 y se puede concluir que el sesgo es diferente de cero. Finalizamos entonces con el último análisis de momento estadístico (curtosis)

In [ ]:
s4=kurtosis(BND)
s4
t4=s4/sqrt(24/T) # Prueba de curtosis, en exceso
t4
pv=2*(1-pnorm(t4)) # p-valor, H0:curtosis=0
pv

La curtosis es positiva, además, el p-valor de esta prueba es menor al alpha de significancia, por tanto se puede rechazar la hipotesis nula, es decir, la curtosis es distinta de cero. Finalmente, se determina si la distribución es o no normal. Podemos usar Jarque Bera o Shapiro. En este caso usamos JB:

In [ ]:
normalTest(BND,method='jb') # Prueba Jaque Bera, H0: Normal

Al aplicar la prueba de normalidad para la distribución se usó el método de Jarque - Bera, esta muestra un p-value menor al valor de significancia del 5%, es decir, la hipótesis nula se rechaza y se puede inferir que la distribución no es normal. Finalmente, se grafica la distribución del bono evaluado.

In [ ]:
library(PerformanceAnalytics)
par(mfrow=c(1,1))
chart.Histogram(BND, methods = c("add.normal", "add.density"), colorset = c("gray", "blue", "red"))
legend("topright", legend = c("Hist-ETF Bond" ,"BND dist","dnorm BND"), col=c("gray", "blue", "red"), lty=1, cex = 0.7)

Así, podemos concluir que el bono ETF evaluado está dominado por un comportamiento leptocúrtico, colas aplanadas en los extremos, donde con una media de rendimientos su distribución es estadísticamente igual a cero. Además, aunque levemente, esta sesgada positivamente y con un exceso de curtosis positivo significativo; lo que se interpreta como un índice atractivo para un inversionista con aversión al riesgo. También se puede explicar esto por ser un bono de una economía estable, así el comportamiento aquí descrito es atractivo para inversión preferente de bajo riesgo, esto dada su poca volatilidad económica en general.

Conclusiones:
Al comparar el S&P 500 y el bono ETF (BND) podemos encontrar de las gráficas de distribución que el BND es una inversión menos riesgosa que el S&P 500, esto dado su sesgo positivo, además se puede soportar este razonamiento en la baja volatilidad mostrada por el BND respecto al S&P 500 en la gráfica de variación de rendimientos. Por tanto, inversionistas aversos al riesgo podrían preferir la inversión en un bono de este tipo (BND) respecto a la inversión en un índice de mercado como el S&P 500. Si bien el S&P 500 no podría catalogarse como un activo altamente riesgoso, dado el comportamiento de sus distribuciones en este intervalo de tiempo se puede considerar una inversión con un perfil de mayor riesgo.
Por otro lado, el S&P 500 muestra una densidad de rendimientos más altos en sus extremos respecto a la opción del BND, es decir la probabilidad de rendimientos altos es mayor en el SPY respecto al comportamiento del BND para la serie evaluada, sin embargo, esto también esta relacionado a mayores perdidas del SPY respecto al BND, por tanto el S&P 500 muestra también un mayor riesgo asociado comparado con el perfil más estable del BND.